In [82]:
from random import shuffle

import pandas as pd
import numpy as np

from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('train.csv')
df.head()

,image,species,individual_id
0,00021adfb725ed.jpg,melon_headed_whale,cadddb1636b9
1,000562241d384d.jpg,humpback_whale,1a71fbb72250
2,0007c33415ce37.jpg,false_killer_whale,60008f293a2b
3,0007d9bca26a99.jpg,bottlenose_dolphin,4b00fe572063
4,00087baf5cef7a.jpg,humpback_whale,8e5253662392


In [3]:
df.describe()

,image,species,individual_id
count,51033,51033,51033
unique,51033,30,15587
top,00021adfb725ed.jpg,bottlenose_dolphin,37c7aba965a5
freq,1,9664,400


In [6]:
df.groupby('individual_id')[['image']].count().sort_values('image', ascending=False)

,image
individual_id,
37c7aba965a5,400
114207cab555,168
a6e325d8e924,155
19fbb960f07d,154
c995c043c353,153
...,...
6a0003ef9501,1
6a05336be085,1
6a0ce195bb3f,1


In [74]:
df_chdt = pd.DataFrame()

for _id in tqdm(df['individual_id'].unique()):
    df_id = df[df['individual_id'] == _id]
    df_id_len = len(df_id)

    df_rest = df[~df['individual_id'].isin([_id])]
    random_ims = df_rest['image'].sample(n=df_id_len).values
    init_ims = df_id['image'].values
    image_b_list = np.concatenate((init_ims, random_ims))

    df_id = df_id.append([df_id], ignore_index=True)
    df_id['image_b'] = image_b_list

    df_chdt = df_chdt.append(df_id)

100%|██████████| 15587/15587 [02:31<00:00, 103.20it/s]


In [79]:
df_chdt.to_csv('df_chdt.csv', index=False)
df_chdt = df_chdt.rename(columns={'image': 'image_a'})
df_chdt.head()

,image_a,species,individual_id,image_b
0,00021adfb725ed.jpg,melon_headed_whale,cadddb1636b9,00021adfb725ed.jpg
1,00021adfb725ed.jpg,melon_headed_whale,cadddb1636b9,6ee7ff58c89139.jpg
0,000562241d384d.jpg,humpback_whale,1a71fbb72250,000562241d384d.jpg
1,ffae18d2939ffc.jpg,humpback_whale,1a71fbb72250,ffae18d2939ffc.jpg
2,000562241d384d.jpg,humpback_whale,1a71fbb72250,2d22eadef9460f.jpg


In [80]:
df_chdt['label'] = df_chdt.apply(lambda x:
                                 1 if x['image_a'] == x['image_b']
                                 else 0, axis=1)

In [88]:
ids = df_chdt['individual_id'].unique().tolist()
shuffle(ids)

train_df = pd.DataFrame()
validate_df = pd.DataFrame()
test_df = pd.DataFrame()

for _id in ids:
    df_id = df_chdt[df_chdt['individual_id'] == _id]
    if train_df.shape[0] <= df_chdt.shape[0] * 0.8:
        train_df = train_df.append(df_id)
    else:
        validate_df = validate_df.append(df_id)
    if validate_df.shape[0] > df_chdt.shape[0] * 0.1:
        test_df = test_df.append(df_id)

train_df.shape, validate_df.shape, test_df.shape

((81658, 5), (20408, 5), (10202, 5))

In [90]:
train_df.to_csv('../csv/train_df.csv', index=False)
validate_df.to_csv('../csv/validate_df.csv', index=False)
test_df.to_csv('../csv/test_df.csv', index=False)